In [2]:
%load_ext autoreload
%autoreload
import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>
    div.output_area{
        max-height:10000px;
        overflow:scroll;
    }
</style>"""))

import uszipcode
import shapefile
from os.path import join
import numpy as np
import pygeohash as geohash
from timeUtils import clock, elapsed
from shapely.geometry.polygon import Polygon
from shapely.geometry import Point
from random import uniform
from geoUtils import *
import folium

basedir="/Users/tgadf/Downloads"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Notebook Last Run Initiated: 2018-08-31 22:13:34.162144


In [ ]:
from collections import Counter
from os.path import basename, dirname, exists, splitext, isdir
from glob import glob
import zipfile
from os import mkdir
from shutil import rmtree

prec=7
shapevals = ["Trans_AirportPoint", "Trans_AirportRunway", "Trans_RailFeature", "Trans_RoadSegment2", "Trans_TrailSegment", "Trans_RoadSegment", "Trans_RoadSegment3"]
shapevals = ["Trans_AirportPoint", "Trans_AirportRunway"]

states = ["11", "17", "24", "51"]
states = ["10"]
states = [str(x) for x in range(20,80)]
basevals = ["TrailSegment", "RailFeature", "RoadSegment"]
show=False
for state in states:
    newdirname = None
    for baseval in basevals:
        shapevals     = ["{0}{1}".format(baseval, x) for x in range(20)]
        shapevals[0]  = baseval
        dirvals       = glob(join(basedir, "stateshapes", "TRAN_{0}_*_GU_STATEORTERRITORY".format(state), "Shape"))
        if len(dirvals) == 0:
            try:
                zipname = glob(join(basedir, "stateshapes", "TRAN_{0}_*_GU_STATEORTERRITORY.zip".format(state)))[0]
            except:
                continue
            if exists(zipname):
                print("Unzipping {0}".format(zipname))
                zip_ref = zipfile.ZipFile(zipname, 'r')
                dirval  = join(join(basedir, "stateshapes", splitext(basename(zipname))[0]))
                if exists(dirval):
                    pass
                else:
                    mkdir(dirval)
                newdirname = dirval
                print("    Unzipping to {0}".format(dirval))
                zip_ref.extractall(dirval)
                zip_ref.close()
                
        
        scntr = Counter()
        remainCntr = Counter()

        shapeData   = {}
        geoShapeMap = {}
        totalgeos   = 0
        keepFrac    = 0
        totalFrac   = 0

        for shapeval in shapevals:
            for dirval in dirvals:
                if newdirname is None:
                    newdirname = dirname(dirval)
                try:
                    sf = shapefile.Reader(join(dirval, "Trans_{0}".format(shapeval)))
                except:
                    continue
                fields = sf.fields
                shapes = sf.shapes()
                Nshapes = len(shapes)
                ngeos = 0

                start,cmt = clock("Analyzing {0}\t{1}\t{2}".format(state, shapeval, Nshapes))
                if show:
                    print("\n\nFields -> {0}".format(fields))

                maxmiss=None
                if shapeval.startswith("Airport"):
                    itype=None
                    for i,val in enumerate(fields):
                        if val[0] == "FAA_AIRPOR":
                            itype = i-1
                            break
                if shapeval.startswith("RailFeature"):
                    maxmiss=50 
                    itype=None
                    for i,val in enumerate(fields):
                        if val[0] == "LENGTHKM":
                            itype = i-1
                            break
                if shapeval.startswith("TrailSegment"):
                    itype=None
                    for i,val in enumerate(fields):
                        if val[0] == "SOURCE_D00":
                            itype = i-1
                            break
                if shapeval.startswith("RoadSegment"):
                    maxmiss=50
                    itype=None
                    for i,val in enumerate(fields):
                        if val[0] == "PERMANENT_":
                            itype = i-1
                            break
                    iint=None
                    for i,val in enumerate(fields):
                        if val[0] == "INTERSTATE":
                            iint = i-1
                            break



                irec = -1
                ngeos = 0
                for shapeRec in sf.iterShapeRecords():
                    totalFrac += 1
                    keep = True
                    irec += 1

                    if totalFrac % 50000 == 0 and totalFrac > 0:
                        print("\tKeeping {0} of {1} = {2}".format(keepFrac, totalFrac, round(100*keepFrac/totalFrac,1)))

                    if show:
                        print(len(shapeRec.shape.points), shapeRec.record)
                        #continue

                    ## Record
                    record = shapeRec.record
                    extra  = []
                    if shapeval.startswith("AirportPoint"):
                        geoid = record[itype]
                        name  = record[itype+1]
                        test  = record[itype+2]
                        try:
                            int(test)
                        except:
                            continue
                    elif shapeval.startswith("AirportRunway"):
                        geoid  = record[itype]
                        name   = record[itype+1]
                    elif shapeval.startswith("RailFeature"):
                        length = record[itype]
                        name   = record[itype+3]
                        geoid  = name
                    elif shapeval.startswith("TrailSegment"):
                        length = record[itype]
                        name   = record[itype+3]
                        geoid  = name
                    elif shapeval.startswith("RoadSegment"):
                        geoid  = record[itype]
                        name   = record[itype+21]
                        inter  = "-".join(record[iint:iint+4])
                        usrte  = "-".join(record[iint+4:iint+8])
                        strte  = "-".join(record[iint+8:iint+12])
                        ctrte  = record[iint+12]
                        street = record[iint+15]
                        scntr[street] += 1
                        extra  = [inter, usrte, strte, ctrte, street]

                        ## Interstate
                        isInterstate = False
                        if inter != '---':
                            isInterstate = True

                        ## USRte
                        isUSRte = False
                        if usrte != '---':
                            isUSRte = True

                        ## StateRte
                        isStateRte = False
                        if strte != '---':
                            isStateRte = True

                        ## Highway
                        isHighway = False
                        try:
                            if any([street.find(" {0}".format(x)) != -1 for x in ['Hw ', 'Hwy', 'Pkwy', 'Hwy', 'Fwy', 'Tollway', 'Expy']]):
                                isHighway = True
                        except:
                            pass

                        ## Major Road
                        isMajorRd = False
                        try:
                            if any([street.find(" {0}".format(x)) != -1 for x in ['Ave', 'Blvd']]):
                                isMajorRd = True
                        except:
                            pass

                        ## Connections
                        isConnection = False
                        try:
                            if any([street.find(" {0}".format(x)) != -1 for x in ['Bridge ', ' Bdg']]):
                                isConnection = True
                        except:
                            pass
                    else:
                        print(record)
                        1/0

                    
                    if shapeval.startswith("TrailSegment"):
                        geos = addLinearGeos(irec, Nshapes, shapeRec.shape, prec, debug=False)
                        ngeos += len(geos)
                        totalgeos += ngeos
                        catID = "Trail"
                        geoid = catID
                        name  = catID
                        if shapeData.get(catID) is None:
                            shapeData[catID] = {}
                            shapeData[catID][geoid]   = {"Name": name}
                            geoShapeMap[catID] = {}
                            geoShapeMap[catID][geoid] = set()
                        for geo in geos:
                            geoShapeMap[catID][geoid].add(geo)

                    
                    if shapeval.startswith("RailFeature"):
                        geos = addLinearGeos(irec, Nshapes, shapeRec.shape, prec, debug=False)
                        ngeos += len(geos)
                        totalgeos += ngeos
                        catID = "Rail"
                        geoid = catID
                        name  = catID
                        if shapeData.get(catID) is None:
                            shapeData[catID] = {}
                            shapeData[catID][geoid]   = {"Name": name}
                            geoShapeMap[catID] = {}
                            geoShapeMap[catID][geoid] = set()
                        for geo in geos:
                            geoShapeMap[catID][geoid].add(geo)


                    if shapeval.startswith("RoadSegment"):                
                        if not any([isInterstate, isUSRte, isStateRte, isHighway, isMajorRd, isConnection]):       
                            continue
                        else:
                            keepFrac += 1

                        geos = addLinearGeos(irec, Nshapes, shapeRec.shape, prec, debug=False)
                        ngeos += len(geos)
                        totalgeos += ngeos
                        if isInterstate:
                            catID = "Interstate"
                            geoid = catID
                            name  = catID
                            if shapeData.get(catID) is None:
                                shapeData[catID] = {}
                                shapeData[catID][geoid]   = {"Name": name}
                                geoShapeMap[catID] = {}
                                geoShapeMap[catID][geoid] = set()
                            for geo in geos:
                                geoShapeMap[catID][geoid].add(geo)
                        if isUSRte:
                            catID = "USRte"
                            geoid = catID
                            name  = catID
                            if shapeData.get(catID) is None:
                                shapeData[catID] = {}
                                shapeData[catID][geoid]   = {"Name": name}
                                geoShapeMap[catID] = {}
                                geoShapeMap[catID][geoid] = set()
                            for geo in geos:
                                geoShapeMap[catID][geoid].add(geo)
                        if isStateRte:
                            catID = "StateRte"
                            geoid = catID
                            name  = catID
                            if shapeData.get(catID) is None:
                                shapeData[catID] = {}
                                shapeData[catID][geoid]   = {"Name": name}
                                geoShapeMap[catID] = {}
                                geoShapeMap[catID][geoid] = set()
                            for geo in geos:
                                geoShapeMap[catID][geoid].add(geo)
                        if isHighway:
                            catID = "Highway"
                            geoid = catID
                            name  = catID
                            if shapeData.get(catID) is None:
                                shapeData[catID] = {}
                                shapeData[catID][geoid]   = {"Name": name}
                                geoShapeMap[catID] = {}
                                geoShapeMap[catID][geoid] = set()
                            for geo in geos:
                                geoShapeMap[catID][geoid].add(geo)
                        if isMajorRd:
                            catID = "MajorRd"
                            geoid = catID
                            name  = catID
                            if shapeData.get(catID) is None:
                                shapeData[catID] = {}
                                shapeData[catID][geoid]   = {"Name": name}
                                geoShapeMap[catID] = {}
                                geoShapeMap[catID][geoid] = set()
                            for geo in geos:
                                geoShapeMap[catID][geoid].add(geo)
                        if isConnection:
                            catID = "Connection"
                            geoid = catID
                            name  = catID
                            if shapeData.get(catID) is None:
                                shapeData[catID] = {}
                                shapeData[catID][geoid]   = {"Name": name}
                                geoShapeMap[catID] = {}
                                geoShapeMap[catID][geoid] = set()
                            for geo in geos:
                                geoShapeMap[catID][geoid].add(geo)

                print("Found {0} geos from {1}-{2}".format(ngeos, state, shapeval))


        for catID,catData in shapeData.items():
            shapeval = catID
            ngeos = len(geoShapeMap[catID][catID])
            print("Found {0} of type {1}/{2}".format(ngeos,baseval,shapeval))
            saveGeoData(shapeData[catID], geoShapeMap[catID], 0, ngeos, "roads/{0}-{1}-{2}-{3}".format(baseval, shapeval, state, prec))
            
    if newdirname is not None:
        if isdir(newdirname):
            print("--->>> Removing {0} directory <<<---".format(newdirname))
            rmtree(newdirname)